In [1]:
import os
from dotenv import load_dotenv

from crewai import Agent, Task, Crew, Process
from crewai_tools import LlamaIndexTool
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent

load_dotenv('../.env')

COLLECTION_NAME = "crewai_demo"
jinaai_api_key = os.environ.get("JINAAI_API_KEY")
groq_api_key = os.environ.get("GROQ_API_KEY")
lgfuse_pub_key = os.environ.get("LANGFUSE_PUBLIC_KEY")
lgfuse_secret_key = os.environ.get("LANGFUSE_SECRET_KEY")

In [2]:
from langchain_openai import ChatOpenAI

chat_llm = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1",
    openai_api_key=groq_api_key,
    model="llama3-70b-8192",
    temperature=0,
    max_tokens=1000,
)

Similar concept as basic RAG

In [3]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core import Settings
from llama_index.llms.groq import Groq
from llama_index.embeddings.jinaai import JinaEmbedding
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.core.callbacks import CallbackManager
from langfuse.llama_index import LlamaIndexCallbackHandler
from llama_index.core import StorageContext



langfuse_callback_handler = LlamaIndexCallbackHandler(
    public_key=lgfuse_pub_key,
    secret_key=lgfuse_secret_key,
    host="http://127.0.0.1:3000"
)


Settings.callback_manager = CallbackManager([langfuse_callback_handler])
Settings.llm = Groq(model="llama3-70b-8192", api_key=groq_api_key)
Settings.embed_model = JinaEmbedding(
    api_key=jinaai_api_key,
    #embed_batch_size=768,
    model="jina-embeddings-v2-base-en",
)

reader = SimpleDirectoryReader(input_files=["./data/uber_10k.pdf"])
documents = reader.load_data()
documents[1]

/home/aizat/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Document(id_='a6806bfb-9a88-4bc6-adc0-2cdcee67e595', embedding=None, metadata={'page_label': '2', 'file_name': 'uber_10k.pdf', 'file_path': 'data/uber_10k.pdf', 'file_type': 'application/pdf', 'file_size': 2829436, 'creation_date': '2024-07-21', 'last_modified_date': '2020-03-31'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='69  \nCountries\n10K+  \nCities\n$65B  \nGross Bookings\n111M  \nMAPCs\n7B  \nTripsA global tech \nplatform at \nmassive scale\nServing multiple multi-trillion \ndollar markets with products \nleveraging our core technology \nand infrastructure\nWe believe deeply in our bold mission. Every minute \nof every day, consumers and Drivers on our platform \ncan tap a button and get a ride or tap a button and \nget work. We revolutioniz

In [4]:
index = VectorStoreIndex.from_documents(documents,)
query_engine = index.as_query_engine(similarity_top_k=5)

2024-07-21 01:24:52,215 - 139696417011264 - task_manager.py-task_manager:137 - WARNING: Item exceeds size limit (size: 1126198), dropping input / output / metadata of item until it fits.


In [5]:
#vector_store = MilvusVectorStore(
#    uri='http://127.0.0.1:19530',
#    collection_name=COLLECTION_NAME,
#    dim=768,
#    overwrite=True,
#    enable_sparse=True,
#    hybrid_ranker="RRFRanker",
#    hybrid_ranker_params={"k": 60},
#)
#storage_context = StorageContext.from_defaults(vector_store=vector_store)
#index = VectorStoreIndex.from_documents(
#    documents, storage_context=storage_context
#)
#query_engine = index.as_query_engine(vector_store_query_mode="hybrid", similarity_top_k=5,)

Load query engine as a query tool for crewai

In [6]:
from crewai_tools import LlamaIndexTool
query_tool = LlamaIndexTool.from_query_engine(
    query_engine,
    name="Uber 2019 10K Query Tool",
    description="Use this tool to lookup the 2019 Uber 10K Annual Report",
)
#
query_tool.args_schema.schema()

{'title': 'QueryToolSchema',
 'description': 'Schema for query tool.',
 'type': 'object',
 'properties': {'query': {'title': 'Query',
   'description': 'Search query for the query tool.',
   'type': 'string'}},
 'required': ['query']}

Define Agents

In [8]:
from crewai import Agent, Task, Crew, Process

# Define your agents with roles and goals
researcher = Agent(
    role="Senior Financial Analyst",
    goal="Uncover insights about different tech companies",
    backstory="""You work at an asset management firm.
  Your goal is to understand tech stocks like Uber.""",
    verbose=True,
    allow_delegation=False,
    tools=[query_tool],
    llm=chat_llm,
)
writer = Agent(
    role="Tech Content Strategist",
    goal="Craft compelling content on tech advancements",
    backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
    llm=chat_llm,
    verbose=True,
    allow_delegation=False,
)

In [9]:
task1 = Task(
    description="""Conduct a comprehensive analysis of Uber's risk factors in 2019.""",
    expected_output="Full analysis report in bullet points",
    agent=researcher,
)

task2 = Task(
    description="""Using the insights provided, develop an engaging blog
  post that highlights the headwinds that Uber faces.
  Your post should be informative yet accessible, catering to a casual audience.
  Make it sound cool, avoid complex words.""",
    expected_output="Full blog post of at least 4 paragraphs",
    agent=writer,
)

In [10]:
crew = Crew(
    agents=[researcher, writer],
    tasks=[task1, task2],
    verbose=2,  # You can set it to 1 or 2 to different logging levels
)

In [11]:
result = crew.kickoff()

 [2024-07-21 01:26:40][DEBUG]: == Working Agent: Senior Financial Analyst
 [2024-07-21 01:26:40][INFO]: == Starting Task: Conduct a comprehensive analysis of Uber's risk factors in 2019.


> Entering new CrewAgentExecutor chain...
Thought: I need to understand the risk factors that Uber faced in 2019.

Action: Uber 2019 10K Query Tool
Action Input: {"query": "Risk Factors"} 

The company faces various risks that could negatively impact its business, financial condition, and results of operations. These risks include:

1. Operational and compliance challenges caused by distance, language, and cultural differences when conducting business internationally, particularly in countries with limited experience.

2. Risks related to laws and regulations that are more restrictive than those in the United States, including laws governing competition, pricing, payment methods, Internet activities, transportation services, logistics services, payment processing and payment gateways, real estate ten